# The JuMP ecosystem for mathematical optimization: Topics

## JuliaCon 2018

## Juan Pablo Vielma
## MIT Sloan

In [1]:
import Pkg
Pkg.activate(@__DIR__)
Pkg.instantiate()

  Updating registry at `~/.julia/registries/General`
  Updating git-repo `https://github.com/JuliaRegistries/General.git`


In [2]:
using JuMP  
using MathOptInterface # Replaces MathProgBase
# shortcuts
const MOI = MathOptInterface
const MOIU = MathOptInterface.Utilities

using GLPK # Loading the GLPK module for using its solver
#using Compat

# Duality

In [3]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x >= 0)
@variable(model, y >= 0)
@constraint(model,inequality, x + y <= 1)         
@objective(model, Max, x + 2y)
JuMP.optimize!(model)
@show JuMP.termination_status(model) == MOI.OPTIMAL
@show JuMP.dual_status(model) == MOI.FEASIBLE_POINT
@show JuMP.dual(inequality)

JuMP.termination_status(model) == MOI.OPTIMAL = true
JuMP.dual_status(model) == MOI.FEASIBLE_POINT = true
JuMP.dual(inequality) = -2.0


-2.0

Also duals for variable bounds.

In [4]:
@show JuMP.dual(JuMP.LowerBoundRef(x))
@show JuMP.dual(JuMP.LowerBoundRef(y))

JuMP.dual(JuMP.LowerBoundRef(x)) = 1.0
JuMP.dual(JuMP.LowerBoundRef(y)) = 0.0


0.0

For sign conventions and precise definition of "dual" problem see [Duality](http://www.juliaopt.org/MathOptInterface.jl/stable/apimanual#Duals-1) in [MOI Manual](http://www.juliaopt.org/MathOptInterface.jl/stable)

# More on JuMP Containers

In [5]:
model = Model(with_optimizer(GLPK.Optimizer))
@variable(model, x[1:5, 1:5])            # Array   <=> @variable(model, x[i=1:5, j=1:5], container = Auto)
set_1 = Base.OneTo(5)
@variable(model, y[set_1, 1:5])          # Array
set_2 = 1:5
@variable(model, z[1:5, set_2])          # DenseAxisArray
@variable(model, z2[i=1:5, j=set_2], container = Array)  # Array
a = 1
@variable(model, zz1[i=1:5, j=a:5])                      # DenseAxisArray
@variable(model, zz2[i=1:5, j=a:5], container = Array)   # Array
set_3 = [:a, :b, :c]
@variable(model, w[set_2, set_3])        # DenseAxisArray
@variable(model, t[i=set_2, 1:i])        # SparseAxisArray
@variable(model, h[i = 1:5; isodd(i)]);  # SparseAxisArray

Auto chooses the tightest applicable container based on compile-time information only and is **type stable**

### `x[i=1:5, j=1:5]`

In [6]:
#@macroexpand @variable(model, x[i=1:5, j=1:5])  # Array
using JuMP.Containers
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(x[i=1:5, j=1:5]))
@show idxvars
@show idxsets
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

idxvars = Any[:i, :j]
idxsets = Any[:($(Expr(:escape, :(1:5)))), :($(Expr(:escape, :(1:5))))]


:((Main.Array){VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(1:5))...))

In [7]:
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, [:i,:j], [:(1:5), :(1:5)], :Auto)

(:(Array{VariableRef}(undef, (length(1:5), length(1:5))...)), true)

### `y[i=set_1, j=1:5], set_1 = Base.OneTo(5)`

In [8]:
#@macroexpand @variable(model, y[i=set_1, j=1:5]) # Array
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(y[i=set_1, j=1:5]))
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_1 isa (Main.Base).OneTo)))
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      (Main.Array){VariableRef}(Main.undef, ((Main.length)(set_1), (Main.length)(1:5))...)
  else
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      ((Main.JuMP).Containers).DenseAxisArray{VariableRef}(Main.undef, set_1, 1:5)
  end)

In [9]:
set_1 isa (Main.Base).OneTo

true

### `z[i=1:5, j=set_2], set_2 = 1:5`

In [10]:
#@macroexpand @variable(model, z1[i=1:5, j=set_2])  # DenseAxisArray
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(set_2 isa (Main.Base).OneTo)))
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      (Main.Array){VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
  else
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      ((Main.JuMP).Containers).DenseAxisArray{VariableRef}(Main.undef, 1:5, set_2)
  end)

In [11]:
set_2 isa (Main.Base).OneTo

false

In [12]:
#@macroexpand @variable(model, z2[i=1:5, j=set_2], container = Array) # Array  
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(z[i=1:5, j=set_2]))
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

quote
    $(Expr(:&&, :(((Main.JuMP).Containers).valid_array_index_set(set_2)))) || (Main.error)("Index set for array is not one-based interval.")
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:116 =#
    (Main.Array){VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(set_2))...)
end

In [13]:
Containers.valid_array_index_set(set_2)

true

In [14]:
@code_typed Containers.valid_array_index_set(set_2)

CodeInfo(
1 ─ %1 = (Base.getfield)(r, :start)::Int64
│   %2 = (%1 === 1)::Bool
└──      return %2
) => Bool

In [15]:
methods(Containers.valid_array_index_set)

# 3 methods for generic function "valid_array_index_set":
[1] valid_array_index_set(::Base.OneTo) in JuMP.Containers at /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:36
[2] valid_array_index_set(r::UnitRange) in JuMP.Containers at /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:37
[3] valid_array_index_set(s) in JuMP.Containers at /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:38

In [43]:
@code_typed Containers.valid_array_index_set(set_1)

CodeInfo(
1 ─     return true
) => Bool

### `zz[i=1:5, j=a:5], a = 1`

In [16]:
#@macroexpand @variable(model, zz1[i=1:5, j=a:5])  # DenseAxisArray
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Auto)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

:(if $(Expr(:&&, :(a:5 isa (Main.Base).OneTo)))
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      (Main.Array){VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(a:5))...)
  else
      #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:142 =#
      ((Main.JuMP).Containers).DenseAxisArray{VariableRef}(Main.undef, 1:5, a:5)
  end)

In [17]:
a:5 isa (Main.Base).OneTo

false

In [18]:
#@macroexpand @variable(m, zz1[i=1:5, j=a:5], container = Array)   # Array
refcall, idxvars, idxsets, condition = JuMP._build_ref_sets(:(zz1[i=1:5, j=a:5]))
containercode, autoduplicatecheck = Containers.generate_container(VariableRef, idxvars, idxsets, :Array)
macro tempmacro()
    containercode
end
@macroexpand @tempmacro 

quote
    $(Expr(:&&, :(((Main.JuMP).Containers).valid_array_index_set(a:5)))) || (Main.error)("Index set for array is not one-based interval.")
    #= /Users/jvielma/.julia/packages/JuMP/jnmGG/src/Containers/generate_container.jl:116 =#
    (Main.Array){VariableRef}(Main.undef, ((Main.length)(1:5), (Main.length)(a:5))...)
end

In [19]:
Containers.valid_array_index_set(a:5)

true

In [20]:
@code_typed Containers.valid_array_index_set(a:5)

CodeInfo(
1 ─ %1 = (Base.getfield)(r, :start)::Int64
│   %2 = (%1 === 1)::Bool
└──      return %2
) => Bool

### Warning: Testing with Containers.valid_array_index_set may **NOT** be **type stable**

In [21]:
function get_my_variables(m,set)
    @variable(m,[set])
end

get_my_variables (generic function with 1 method)

In [22]:
function get_my_variables_with_testing(m,set)
    if Containers.valid_array_index_set(set)
        return @variable(m,[set],container=Array)
    else
        return @variable(m,[set])
    end
end

get_my_variables_with_testing (generic function with 1 method)

Type stable:

In [23]:
@code_warntype get_my_variables_with_testing(model,set_1)

Body::Array{VariableRef,1}
1 ──       nothing
│    %2  = (Base.getfield)(set, :stop)::Int64
│    %3  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{VariableRef,1}, svec(Any, Int64), :(:ccall), 2, Array{VariableRef,1}, :(%2), :(%2)))::Array{VariableRef,1}
│    %4  = (Base.getfield)(set, :stop)::Int64
│    %5  = (Base.slt_int)(%4, 1)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ──       goto #4
4 ┄─ %9  = φ (#2 => true, #3 => false)::Bool
│    %10 = φ (#3 => 1)::Int64
│    %11 = φ (#3 => 1)::Int64
│    %12 = (Base.not_int)(%9)::Bool
└───       goto #20 if not %12
5 ┄─ %14 = φ (#4 => %10, #19 => %38)::Int64
│    %15 = φ (#4 => %11, #19 => %39)::Int64
│    %16 = (Base.sle_int)(1, 1)::Bool
└───       goto #7 if not %16
6 ── %18 = (Base.sle_int)(1, 0)::Bool
└───       goto #8
7 ──       nothing
8 ┄─ %21 = φ (#6 => %18, #7 => false)::Bool
└───       goto #10 if not %21
9 ──       invoke Base.getindex(()::Tuple, 1::Int64)
└───       $(Expr(:unreachable))
10 ┄       goto #11
11 

Not type stable:

In [24]:
@code_warntype get_my_variables_with_testing(model,set_2)

Body::Union{DenseAxisArray{VariableRef,1,Tuple{UnitRange{Int64}},Tuple{Dict{Int64,Int64}}}, Array{VariableRef,1}}
1 ── %1   = (Base.getfield)(set, :start)::Int64
│    %2   = (%1 === 1)::Bool
└───        goto #31 if not %2
2 ── %4   = (Base.getfield)(set, :start)::Int64
│    %5   = (%4 === 1)::Bool
└───        goto #30 if not %5
3 ──        nothing
│    %8   = (Base.getfield)(set, :stop)::Int64
│    %9   = (Base.getfield)(set, :start)::Int64
│    %10  = (Base.Checked.checked_ssub_int)(%8, %9)::Tuple{Int64,Bool}
│    %11  = (Base.getfield)(%10, 1)::Int64
│    %12  = (Base.getfield)(%10, 2)::Bool
└───        goto #5 if not %12
4 ──        invoke Base.Checked.throw_overflowerr_binaryop(:-::Symbol, %8::Int64, %9::Int64)
└───        $(Expr(:unreachable))
5 ┄─        goto #6
6 ── %17  = (Base.Checked.checked_sadd_int)(%11, 1)::Tuple{Int64,Bool}
│    %18  = (Base.getfield)(%17, 1)::Int64
│    %19  = (Base.getfield)(%17, 2)::Bool
└───        goto #8 if not %19
7 ──        invoke Base.Checked.th

Type stable:

In [25]:
@code_warntype get_my_variables_with_testing(model,set_3)

Body::DenseAxisArray{VariableRef,1,Tuple{Array{Symbol,1}},Tuple{Dict{Symbol,Int64}}}
1 ──       goto #3 if not false
2 ──       nothing
3 ┄─       goto #5 if not false
4 ──       nothing
5 ┄─ %5  = (Base.arraylen)(set)::Int64
│    %6  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{VariableRef,1}, svec(Any, Int64), :(:ccall), 2, Array{VariableRef,1}, :(%5), :(%5)))::Array{VariableRef,1}
│    %7  = (Core.tuple)(set)::Tuple{Array{Symbol,1}}
│    %8  = invoke JuMP.Containers.build_lookup(_3::Array{Symbol,1})::Dict{Symbol,Int64}
│    %9  = (Core.tuple)(%8)::Tuple{Dict{Symbol,Int64}}
│    %10 = %new(DenseAxisArray{VariableRef,1,Tuple{Array{Symbol,1}},Tuple{Dict{Symbol,Int64}}}, %6, %7, %9)::DenseAxisArray{VariableRef,1,Tuple{Array{Symbol,1}},Tuple{Dict{Symbol,Int64}}}
│    %11 = (Base.arraylen)(set)::Int64
│    %12 = (Base.sle_int)(0, %11)::Bool
│    %13 = (Base.bitcast)(UInt64, %11)::UInt64
│    %14 = (Base.ult_int)(0x0000000000000000, %13)::Bool
│    %15 = (Base.and_int)(%12, %14)::Bo

**Note:** `@variable` should always be type stable: 

In [26]:
function get_my_variables_array(m,set)
    @variable(m,[set], container=Array)
end

get_my_variables_array (generic function with 1 method)

In [27]:
@code_warntype get_my_variables_array(model,set_1)

Body::Array{VariableRef,1}
1 ──       nothing
│    %2  = (Base.getfield)(set, :stop)::Int64
│    %3  = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{VariableRef,1}, svec(Any, Int64), :(:ccall), 2, Array{VariableRef,1}, :(%2), :(%2)))::Array{VariableRef,1}
│    %4  = (Base.getfield)(set, :stop)::Int64
│    %5  = (Base.slt_int)(%4, 1)::Bool
└───       goto #3 if not %5
2 ──       goto #4
3 ──       goto #4
4 ┄─ %9  = φ (#2 => true, #3 => false)::Bool
│    %10 = φ (#3 => 1)::Int64
│    %11 = φ (#3 => 1)::Int64
│    %12 = (Base.not_int)(%9)::Bool
└───       goto #20 if not %12
5 ┄─ %14 = φ (#4 => %10, #19 => %38)::Int64
│    %15 = φ (#4 => %11, #19 => %39)::Int64
│    %16 = (Base.sle_int)(1, 1)::Bool
└───       goto #7 if not %16
6 ── %18 = (Base.sle_int)(1, 0)::Bool
└───       goto #8
7 ──       nothing
8 ┄─ %21 = φ (#6 => %18, #7 => false)::Bool
└───       goto #10 if not %21
9 ──       invoke Base.getindex(()::Tuple, 1::Int64)
└───       $(Expr(:unreachable))
10 ┄       goto #11
11 

In [28]:
@code_warntype get_my_variables_array(model,set_2)

Body::Array{VariableRef,1}
1 ── %1  = (Base.getfield)(set, :start)::Int64
│    %2  = (%1 === 1)::Bool
└───       goto #29 if not %2
2 ──       nothing
│    %5  = (Base.getfield)(set, :stop)::Int64
│    %6  = (Base.getfield)(set, :start)::Int64
│    %7  = (Base.Checked.checked_ssub_int)(%5, %6)::Tuple{Int64,Bool}
│    %8  = (Base.getfield)(%7, 1)::Int64
│    %9  = (Base.getfield)(%7, 2)::Bool
└───       goto #4 if not %9
3 ──       invoke Base.Checked.throw_overflowerr_binaryop(:-::Symbol, %5::Int64, %6::Int64)
└───       $(Expr(:unreachable))
4 ┄─       goto #5
5 ── %14 = (Base.Checked.checked_sadd_int)(%8, 1)::Tuple{Int64,Bool}
│    %15 = (Base.getfield)(%14, 1)::Int64
│    %16 = (Base.getfield)(%14, 2)::Bool
└───       goto #7 if not %16
6 ──       invoke Base.Checked.throw_overflowerr_binaryop(:+::Symbol, %8::Int64, 1::Int64)
└───       $(Expr(:unreachable))
7 ┄─       goto #8
8 ──       goto #9
9 ── %22 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{VariableRef,1}, svec(Any, I

In [29]:
@code_warntype get_my_variables_array(model,set_3)

Body::Union{}
1 ─     goto #3 if not false
2 ─     nothing
3 ┄     invoke JuMP.error("Index set for array is not one-based interval."::String)
└──     $(Expr(:unreachable))


# More Type Instability Warnings

cf. https://github.com/JuliaOpt/JuMP.jl/pull/1348

In [30]:
using ECOS
model = Model(with_optimizer(ECOS.Optimizer))
@variable(model, x[i=1:5,j=1:5]) 

5×5 Array{VariableRef,2}:
 x[1,1]  x[1,2]  x[1,3]  x[1,4]  x[1,5]
 x[2,1]  x[2,2]  x[2,3]  x[2,4]  x[2,5]
 x[3,1]  x[3,2]  x[3,3]  x[3,4]  x[3,5]
 x[4,1]  x[4,2]  x[4,3]  x[4,4]  x[4,5]
 x[5,1]  x[5,2]  x[5,3]  x[5,4]  x[5,5]

In [31]:
@constraint(model,[i=1:size(x,1)], x[i,:] in MOI.SecondOrderCone(5))

5-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 [x[1,1], x[1,2], x[1,3], x[1,4], x[1,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[2,1], x[2,2], x[2,3], x[2,4], x[2,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[3,1], x[3,2], x[3,3], x[3,4], x[3,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[4,1], x[4,2], x[4,3], x[4,4], x[4,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[5,1], x[5,2], x[5,3], x[5,4], x[5,5]] ∈ MathOptInterface.SecondOrderCone(5)

In [32]:
function add_my_constraints(model,x)
    n,m = size(x)
    @constraint(model,[i=1:n], x[i,:] in MOI.SecondOrderCone(m))
end

add_my_constraints (generic function with 1 method)

In [33]:
add_my_constraints(model,x)

5-element Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}:
 [x[1,1], x[1,2], x[1,3], x[1,4], x[1,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[2,1], x[2,2], x[2,3], x[2,4], x[2,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[3,1], x[3,2], x[3,3], x[3,4], x[3,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[4,1], x[4,2], x[4,3], x[4,4], x[4,5]] ∈ MathOptInterface.SecondOrderCone(5)
 [x[5,1], x[5,2], x[5,3], x[5,4], x[5,5]] ∈ MathOptInterface.SecondOrderCone(5)

In [34]:
@code_warntype add_my_constraints(model,x)

Body::Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}
1 ── %1  = (Base.arraysize)(x, 1)::Int64
│    %2  = (Base.arraysize)(x, 2)::Int64
│    %3  = (Base.sle_int)(1, %1)::Bool
│    %4  = (Base.ifelse)(%3, %1, 0)::Int64
│    %5  = (Base.Checked.checked_ssub_int)(%4, 1)::Tuple{Int64,Bool}
│    %6  = (Base.getfield)(%5, 1)::Int64
│    %7  = (Base.getfield)(%5, 2)::Bool
└───       goto #3 if not %7
2 ──       invoke Base.Checked.throw_overflowerr_binaryop(:-::Symbol, %4::Int64, 1::Int64)
└───       $(Expr(:unreachable))
3 ┄─       goto #4
4 ── %12 = (Base.Checked.checked_sadd_int)(%6, 1)::Tuple{Int64,Bool}
│    %13 = (Base.getfield)(%12, 1)::Int64
│    %14 = (Base.getfield)(%12, 2)::Bool
└───       goto #6 if not %14
5 ──       invoke Base.Checked.throw_overflowerr_binaryop(:+::Symbol, %6::Int64, 1::Int64)
└───       $(Expr(:unreachable))
6 ┄─       goto #7
7 ──       goto #8
8 ── %20 = $(Expr(:foreigncall, :(:jl_alloc_array_1d), Array{ConstraintRef{Model,C,Shape} w

In [35]:
function add_my_constraints_type(model,x,const_type)
    n,m = size(x)
    constraints = (x -> x ? MOI.SecondOrderCone(m) : MOI.RotatedSecondOrderCone(m)).(const_type)
    @constraint(model,[i=1:n], x[i,:] in constraints[i])
end

add_my_constraints_type (generic function with 1 method)

In [36]:
const_type = rand([true,false],5)

5-element Array{Bool,1}:
 false
 false
  true
  true
 false

In [37]:
@code_warntype add_my_constraints_type(model,x,const_type)

Body::Array{ConstraintRef{Model,C,Shape} where Shape<:AbstractShape where C,1}
1 ── %1   = (Base.arraysize)(x, 1)::Int64
│    %2   = (Base.arraysize)(x, 2)::Int64
│    %3   = %new(getfield(Main, Symbol("##3#4")){Int64}, %2)::getfield(Main, Symbol("##3#4")){Int64}
│    %4   = (Core.tuple)(const_type)::Tuple{Array{Bool,1}}
│    %5   = (Base.arraysize)(const_type, 1)::Int64
│    %6   = (Base.slt_int)(%5, 0)::Bool
│    %7   = (Base.ifelse)(%6, 0, %5)::Int64
│    %8   = %new(Base.OneTo{Int64}, %7)::Base.OneTo{Int64}
│    %9   = (Core.tuple)(%8)::Tuple{Base.OneTo{Int64}}
│    %10  = %new(Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1},Tuple{Base.OneTo{Int64}},getfield(Main, Symbol("##3#4")){Int64},Tuple{Array{Bool,1}}}, %3, %4, %9)::Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1},Tuple{Base.OneTo{Int64}},getfield(Main, Symbol("##3#4")){Int64},Tuple{Array{Bool,1}}}
│    %11  = invoke Base.Broadcast.combine_eltypes(%3::Function, %4::Tuple{Array{Bool,1}})::Type{#s7

│    %142 = (JuMP.build_constraint)(getfield(JuMP, Symbol("#_error#55")){Symbol}(Core.Box(Any[:model, :([i = 1:n]), :(x[i, :] in constraints[i])]), :constraint), %140, %141)::VectorConstraint{VariableRef,_1,VectorShape} where _1
│    %143 = (JuMP.add_constraint)(model, %142, "")::ConstraintRef{Model,_1,VectorShape} where _1
│           (Base.arrayset)(true, %102, %143, %114)
│    %145 = (%115 === %104)::Bool
└───        goto #49 if not %145
48 ─        goto #50
49 ─ %148 = (Base.add_int)(%115, 1)::Int64
└───        goto #50
50 ┄ %150 = φ (#49 => %148)::Int64
│    %151 = φ (#49 => %148)::Int64
│    %152 = φ (#48 => true, #49 => false)::Bool
│    %153 = (Base.not_int)(%152)::Bool
└───        goto #52 if not %153
51 ─        goto #40
52 ┄        return %102
